# <span style="color:green; font-family:Georgia; font-size:2.1m;"> 1- Modèle de prediction 1

# <span style="color:green; font-family:Georgia; font-size:1.5m;"> 1- Préparation des données

In [1]:
import numpy as np
from unidecode import unidecode
import string
from sklearn import svm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mamdiallo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
fichier = open("CorpusM2-AFD/corpus.tache1.learn","r",encoding = "latin-1")
lignes = fichier.readlines()

In [3]:
data = []
for ligne in lignes:
    ligne = ligne.replace('\n','')
    data.append(ligne)

In [4]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
print(data[0])
print(data[0].split(' ')[3])
print(stemmer.stem(data[0].split(' ')[3]))
print(stemmer.stem('diplomatique'))

<100:1:C> Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.
dis
dis
diplomat


In [5]:

print(stopwords)

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

In [6]:
def zeroOuUn(c):
    if c == 'C':
        return 1
    else:
        return 0

In [7]:
def recupPersonne(s):
    for i in range(0,len(s)):
        if s[i]=='>':
            return(zeroOuUn(s[i-1]))

In [8]:
def recupDonnees():
    X = []
    y = []
    for doc in data:
        doc1 = doc.split(' ')
    #     doc1 =  [stemmer.stem(m) for m in doc1] #lemmetisation
    #     print(" ".join(doc1[1:]))
        res = " ".join(doc1[1:])        
        res = res.lower() #convertit les Majuscules en miniscules
        res = unidecode(res) #suppression des accents et cedilles
        res = res.translate(str.maketrans('', '', string.punctuation)) #suprression des ponctuations
        res = res.split(' ')
        res = " ".join([mot for mot in res if mot not in stopwords]) #suppression des stopwords
        X.append(res)
        y.append(recupPersonne(doc1[0]))
        
    return X,y

In [9]:
X, y = recupDonnees()

In [10]:
print(len(X))
print(len(y))

57413
57413


In [11]:
print(y[0], " - ",X[0])
print(y[1], " - ",X[1])
print(y[12], " - ",X[12])

1  -  quand dis chers amis sagit dune formule diplomatique lexpression ressens
1  -  dabord merci cet exceptionnel accueil congolais brazavillois reserve cet apresmidi
0  -  cest vrai quil a tres souvent pays refus regarder droit devant soi comme souhait den rester certaine peur changement cest vrai


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(len(X_train))
print(len(y_train))

print(len(X_test))
print(len(y_test))

print(type(X_train))
print(type(y_train))

print(type(X_test))
print(type(y_test))

45930
45930
11483
11483
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [13]:
mot_max = 2500
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(min_df=5, max_df=0.7)
vectorizer = TfidfVectorizer(max_features=mot_max, min_df=5, max_df=0.7)
X_train_tfifd = vectorizer.fit_transform(X_train)
X_test_tfifd = vectorizer.fit_transform(X_test)

In [14]:
# print(vectorizer.get_feature_names())

In [15]:
print(type(X_train_tfifd.toarray()))
print(type(X_test_tfifd))

<class 'numpy.ndarray'>
<class 'scipy.sparse.csr.csr_matrix'>


In [16]:
print(X_train[0])
print(X_train_tfifd.toarray()[970])


merite attention appui
[0. 0. 0. ... 0. 0. 0.]


In [17]:
print(X_train_tfifd.shape)

(45930, 2500)


In [18]:
import torch
import torch.utils.data as utils
from torchvision import datasets, transforms
import torch.nn.functional as F

In [19]:
torch.set_default_tensor_type(torch.FloatTensor)
# print(torch.get_default_dtype())
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data_tensor = torch.tensor(X_train_tfifd.toarray())
train_target_tensor = torch.tensor(np.asarray(y_train)) # transform to torch tensors

test_data_tensor = torch.tensor(X_test_tfifd.toarray()) # transform to torch tensors
test_target_tensor = torch.tensor(np.asarray(y_test)) # transform to torch tensors

train_dataset = utils.TensorDataset(train_data_tensor,train_target_tensor) # create your datset
test_dataset = utils.TensorDataset(test_data_tensor,test_target_tensor) # create your datset

batch_size = 64
# Set the training loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Set the testing loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [20]:
weights1 = torch.randn(mot_max, 15, requires_grad=True)
weights2 = torch.randn(15, 15, requires_grad=True)
weights3 = torch.randn(15, 2, requires_grad=True)

b1 = torch.randn((1, 15), requires_grad=True) # bias for hidden layer
b2 = torch.randn((1, 15), requires_grad=True) # bias for output layer
b3 = torch.randn((1, 2), requires_grad=True) # bias for output layer

# b1 = torch.randn(500,)
# b2 = torch.randn((1, 2))

In [21]:
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

In [22]:
def test(weights1, weights2,weights3,b1, b2, b3, test_loader):
    test_size = len(test_loader.dataset)
    correct = 0

    for batch_idx, (data, target) in enumerate(test_loader):
        #print(batch_idx, data.shape, target.shape)
#         data = data.view((-1, 28*28))
        #print(batch_idx, data.shape, target.shape)
        data = data.float()
        target = target.to(dtype=torch.long)        
        z1 = torch.matmul(data, weights1) + b1
        a1 = sigmoid_activation(z1)
        
        z2 = torch.matmul(a1, weights2) + b2
        a2 = sigmoid_activation(z2)

        z3 = torch.matmul(a2, weights3) + b3
        
        outputs = sigmoid_activation(z3)
        
        softmax = F.softmax(outputs, dim=1)
        pred = softmax.argmax(dim=1, keepdim=True)
        n_correct = pred.eq(target.view_as(pred)).sum().item()
        correct += n_correct

    acc = correct / test_size
    print(" Accuracy on test set", acc)
    return

test(weights1, weights2,weights3,b1, b2, b3, test_loader)

 Accuracy on test set 0.6771749542802403


In [23]:
it = 0
for batch_idx, (data, targets) in enumerate(train_loader):
    # Be sure to start the loop with zeros grad
    if weights1.grad is not None:
        weights1.grad.zero_()
    
    if weights2.grad is not None:
        weights2.grad.zero_()
        
    if weights3.grad is not None:
        weights3.grad.zero_()
    
    if b1.grad is not None:
        b1.grad.zero_()
        
    if b2.grad is not None:
        b2.grad.zero_()
        
    if b3.grad is not None:
        b3.grad.zero_()
        
        
    #print("batch_idx: {}, data.shape: {}, target.shape: {}".format(batch_idx, data.shape, targets.shape))
    data = data.float()
    targets = targets.to(dtype=torch.long)
    z1 = torch.matmul(data, weights1) + b1
    a1 = sigmoid_activation(z1)
        
    z2 = torch.matmul(a1, weights2) + b2
    a2 = sigmoid_activation(z2)

    z3 = torch.matmul(a2, weights3) + b3
        
    outputs = sigmoid_activation(z3)
    log_softmax = F.log_softmax(outputs, dim=1)
    #print("Log softmax: {}".format(log_softmax.shape))

    #print((-log_softmax[0][targets[0]] + -log_softmax[1][targets[1]] )  / 2 )
    #print(-log_softmax[0][targets[0]], targets[0])
    
    loss = F.nll_loss(log_softmax, targets)
    print("\rLoss shape: {}".format(loss), end="")
    
    # Compute the gradients for each variables
    loss.backward()
    
    with torch.no_grad():
        weights3 -= 0.1*weights3.grad
        weights2 -= 0.1*weights2.grad
        weights1 -= 0.1*weights1.grad
        b3 -= 0.1*b3.grad
        b2 -= 0.1*b2.grad
        b1 -= 0.1*b1.grad


    it += 1
    if it % 10 == 0:
        test(weights1, weights2,weights3,b1, b2, b3, test_loader)
        
#     if it > 50000:
#         break

Loss shape: 0.6479528546333313 Accuracy on test set 0.8400243838718106
Loss shape: 0.5965312123298645 Accuracy on test set 0.8666724723504311
Loss shape: 0.5599048137664795 Accuracy on test set 0.8685012627362187
Loss shape: 0.5551152229309082 Accuracy on test set 0.8685883479926848
Loss shape: 0.5301421284675598 Accuracy on test set 0.8685883479926848
Loss shape: 0.50790590047836343 Accuracy on test set 0.8685883479926848
Loss shape: 0.55523848533630375 Accuracy on test set 0.8685883479926848
Loss shape: 0.46998408436775213 Accuracy on test set 0.8685883479926848
Loss shape: 0.47692486643791265 Accuracy on test set 0.8685883479926848
Loss shape: 0.46575838327407837 Accuracy on test set 0.8685883479926848
Loss shape: 0.43899309635162354 Accuracy on test set 0.8685883479926848
Loss shape: 0.43734255433082587 Accuracy on test set 0.8685883479926848
Loss shape: 0.48982185125350955 Accuracy on test set 0.8685883479926848
Loss shape: 0.51422810554504495 Accuracy on test set 0.86858834799268

In [27]:
batch_idx, (data, target) = next(enumerate(test_loader))
data = data.float()
# for i in data[0]:
#     print(i)
print(target)
z1 = torch.matmul(data, weights1) + b1
a1 = sigmoid_activation(z1)       
z2 = torch.matmul(a1, weights2) + b2
a2 = sigmoid_activation(z2)
z3 = torch.matmul(a2, weights3) + b3      
outputs = sigmoid_activation(z3)
softmax = F.softmax(outputs, dim=1)
pred = softmax.argmax(dim=1, keepdim=True)
print(pred)

tensor([1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1], dtype=torch.int32)
tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1